In [1]:
import stac_ipyleaflet
import ipywidgets

# Use ipyleaflet to visually explore MAAP data

Inspired by Leafmap.

In [2]:
m = stac_ipyleaflet.StacIpyleaflet(layout=ipywidgets.Layout(height="900px"))
m

Output()

StacIpyleaflet(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…